# Análise exploratória de dados espaciais - AEDE

<br>
Municípios grandes produtores de alguma cultura tendem a estar próximos de municípios também produtores dessa cultura ou de municípios que não o produzem? O objetivo da análise exploratória de dados espaciais (AEDE) é responder a esse tipo de questão. De forma mais geral, a questão é qual a a associação entre o valor de certa variável em um lugar e os valores dessa mesma variável nos lugares vizinhos. Localidades com valores semelhantes de determinada variável tendem a estar próximas? Para isso, ela mede a autocorrelação espacial, por meio do $I$ de Moran.

In [ ]:
!pip install geopandas==0.8.2
!pip install --upgrade pyshp
!pip install shapely  ==1.7.0
!pip install --upgrade descartes
!pip install mapclassify==2.3.0 libpysal==4.3.0 splot==1.1.3
!pip install esda
!pip install pysal

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as sm

# para gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# para a análise de dados espaciais
import geopandas as gp
import pysal as ps
import splot
import mapclassify as mc
from libpysal.weights import Queen
from libpysal import weights
from esda import Moran, Moran_Local, G_Local
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation

# links com arquivos úteis
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/'
link_p = 'https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/'

In [ ]:
# ler dados de interesse
dados = pd.read_csv(link_p+'qtde-22.csv', encoding='latin1')
# quantidade produzida em toneladas em 2017

# para ler um arquivo salvo no computador
# ir primeiro em > e Files
# dados = pd.read_csv('qtde_22.csv', encoding='latin1')

In [ ]:
# nomes das variáveis
dados.columns

In [ ]:
# número de linhas e colunas do dataframe
dados.shape

In [ ]:
# mostrar algumas linhas do dataframe
dados.head(3)

In [ ]:
# renomear colunas
# 'ibge7' será 'mun' (código do município)
# 'mun' será 'município' (nome do município)

dados.rename(columns={'mun': 'municipio'}, inplace=True)  # inplace = True modifica o dataframe
dados.rename(columns={'ibge7': 'mun'}, inplace=True)

In [ ]:
# ler shapefile
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/mg.json'
geodf = gp.read_file(link)

In [ ]:
geodf.head()

In [ ]:
# mesclar shapefile com dataframe
# usando a coluna mun
geodf.rename(columns={'CD_GEOCMU': 'mun'}, inplace=True)  # mudar nome da coluna com cód. para 'mun'
geodf['mun'] = geodf.mun.astype(int)  # transformar códigos em inteiros
mg = pd.merge(geodf, dados, on='mun', suffixes=('', '_y'))  # mesclar o dataframe e o shapefile
mg.shape

In [ ]:
# ver objeto resultante (dataframe + shapefile)
mg.head(3)

In [ ]:
mg.plot();

In [ ]:
mg.columns

In [ ]:
variavel = 'soja'

In [ ]:
# mapa temático - simples
mg.plot(variavel);

Esquemas de cores: cmap (https://matplotlib.org/stable/tutorials/colors/colormaps.html)

- PuBu

- GnBu

- Oranges

- BuGn

- Purples

- YlOrBr


In [ ]:
# mapa temático - com opções
# scheme='Quantiles' ou 'Equal_Interval'
# quantiles: atribui mesma quantidade de valores para cada categoria. Apesar de esconder valores extremos,
# pode se tornar mais informativa se a distribuição for assimétrica
# perceber as diferenças entre os esquemas: cmap='PuBu', por exemplo


mg.plot(column=variavel, figsize=(15, 13), scheme='quantiles', legend=True, k=4, cmap='Oranges');

In [ ]:
# mapa temático - com opções
# scheme='equal_interval', 'fisher_jenks'
mg.plot(column=variavel, figsize=(15, 13), scheme='fisher_jenks', legend=True, k=4, cmap='Oranges');

In [ ]:
# mapa temático - com opções
# escolher diferentes esquemas de cores
mg.plot(column=variavel, figsize=(15, 13), scheme='equal_interval', legend=True, k=4, cmap='Oranges');

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(mg)
w.transform = 'r'

# I de Moran

Estatística mais utilizada para medir a autocorrelação espacial. Ela mede a relação do desvio padronizado de uma variável numa área com o desvio padronizado das áreas vizinhas para a mesma variável:

$$I=\frac{N}{S_{0}}\frac{\displaystyle\sum_{i=1}^{n}\sum_{j=1}^{n}w_{ij}z_{i}z_{j}}{\displaystyle\sum_{i=1}^{n}z^{2}_{i}},$$
em que:

- $z_i = y_i - \bar{y}$ representa o desvio em relação à média da variável.
- $y_i$: valor da variável em um determinado local $i$
- $N$: número de observações
- $S_0 = \sum\sum w_{ij}$
- $E[I] = -1/(N - 1) \approx 0$
            
Se o valor-$p$ referente ao teste do $I$ de Moran for significativo, podemos olhar para o valor da estatística $I$ e concluir:

- $I > 0$: autocorrelação espacial positiva (*clusters* espaciais - HH, LL)
- $I < 0$: autocorrelação espacial negativa (*outliers* espaciais - HL, LH)

In [ ]:
# calcular I de Moran global para a variável escolhida
y = mg[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p
moran.p_sim

In [ ]:
# diagrama de dispersão de Moran
plot_moran(moran, zstandard=False, figsize=(10,4));

In [ ]:
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

### LISA

- Estatística para detectar padrões locais de autocorrelação espacial: *Local Indicator of Spatial Association* (LISA), ou $I$ de Moran local  
- Permite verificar se há agrupamentos espaciais estatisticamente significativos
- Útil quando a estatística $I$ de Moran global for significativa

$$I_{i}=z_{i}\sum_{j=1}^{j}w_{ij}y_{j}$$  

In [ ]:
lisa_cluster(moran_loc, mg, p=0.05, figsize = (9,9));

In [ ]:
plot_local_autocorrelation(moran_loc, mg, variavel);